In [6]:
import numpy as np
import random
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
import json
import pickle
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

In [9]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [10]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        
        
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [11]:
documents

[(['Hi', 'there'], 'greeting'),
 (['How', 'are', 'you'], 'greeting'),
 (['Is', 'anyone', 'there', '?'], 'greeting'),
 (['Hey'], 'greeting'),
 (['Hola'], 'greeting'),
 (['Hello'], 'greeting'),
 (['Good', 'day'], 'greeting'),
 (['Bye'], 'goodbye'),
 (['See', 'you', 'later'], 'goodbye'),
 (['Goodbye'], 'goodbye'),
 (['Nice', 'chatting', 'to', 'you', ',', 'bye'], 'goodbye'),
 (['Till', 'next', 'time'], 'goodbye'),
 (['Who', 'is', 'the', 'coordinator', 'of', 'MBD', 'Master', 'program', '?'],
  'coordinator'),
 (['What', "'s", 'the', 'name', 'of', 'the', 'coordinator', '?'],
  'coordinator'),
 (['Did',
   'you',
   'know',
   'the',
   'coordinator',
   'of',
   'this',
   'master',
   'program',
   '?'],
  'coordinator'),
 (['coordintor', 'name'], 'coordinator'),
 (['Thanks'], 'thanks'),
 (['Thank', 'you'], 'thanks'),
 (['That', "'s", 'helpful'], 'thanks'),
 (['Awesome', ',', 'thanks'], 'thanks'),
 (['Thanks', 'for', 'helping', 'me'], 'thanks'),
 (['I', 'appreciate', 'your', 'help', ',', 't

In [12]:
words=[lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
#Eliminate duplicated words
words=sorted(list(set(words)))

In [13]:
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words
print (len(words), "unique lemmatized words", words)
#Serialize lists in disk using pickle 
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

54 documents
11 classes ['CareerOpportunities', 'applying', 'bootname', 'classsize', 'coordinator', 'goodbye', 'greeting', 'keycourses', 'options', 'subjects', 'thanks']
91 unique lemmatized words ["'s", ',', '.', 'after', 'and', 'anyone', 'application', 'apply', 'applying', 'appreciate', 'are', 'awesome', 'be', 'bye', 'can', 'career', 'chatting', 'class', 'condition', 'coordinator', 'coordintor', 'could', 'course', 'day', 'did', 'do', 'each', 'eligible', 'example', 'exists', 'for', 'good', 'goodbye', 'graduation', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'i', 'in', 'is', 'job', 'key', 'know', 'later', 'major', 'many', 'master', 'mater', 'mbd', 'me', 'module', 'name', 'next', 'nice', 'number', 'of', 'offered', 'opportunity', 'outlet', 'please', 'possible', 'program', 'programmed', 'provide', 'requirement', 'see', 'semester', 'size', 'student', 'subject', 'support', 'tell', 'thamks', 'thank', 'thanks', 'that', 'the', 'there', 'this', 'till', 'time', 'to', 'what

In [14]:
len(classes)

11

In [15]:
# create our training data
training=[]
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]

    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")
output_row
training

Training data created


<ipython-input-15-a41ab7351c71>:23: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)


array([[list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
        list([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0])],
       [list([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
        list([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0])],
       [list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]),
        list([0, 0, 1, 0, 0, 0, 0

In [16]:
train_y

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0,

In [17]:
from tensorflow.keras.optimizers import SGD

# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
# Compile model. Stochastic gradient descent 
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("model created")

/Users/achraf/opt/anaconda3/lib/python3.8/site-packages/keras/optimizer_v2/optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/200
11/11 [==============================] - 1s 2ms/step - loss: 2.4034 - accuracy: 0.1667
Epoch 2/200
11/11 [==============================] - 0s 1ms/step - loss: 2.4254 - accuracy: 0.1667
Epoch 3/200
11/11 [==============================] - 0s 2ms/step - loss: 2.2744 - accuracy: 0.2037
Epoch 4/200
11/11 [==============================] - 0s 2ms/step - loss: 2.2819 - accuracy: 0.2593
Epoch 5/200
11/11 [==============================] - 0s 2ms/step - loss: 2.2406 - accuracy: 0.2778
Epoch 6/200
11/11 [==============================] - 0s 2ms/step - loss: 2.2210 - accuracy: 0.2407
Epoch 7/200
11/11 [==============================] - 0s 2ms/step - loss: 2.1806 - accuracy: 0.2778
Epoch 8/200
11/11 [==============================] - 0s 2ms/step - loss: 2.0811 - accuracy: 0.3704
Epoch 9/200
11/11 [==============================] - 0s 1ms/step - loss: 2.0937 - accuracy: 0.3519
Epoch 10/200
11/11 [==============================] - 0s 2ms/step - loss: 2.0193 - accuracy: 0.4074
Epoch 11/

11/11 [==============================] - 0s 1ms/step - loss: 0.0979 - accuracy: 1.0000
Epoch 84/200
11/11 [==============================] - 0s 2ms/step - loss: 0.0838 - accuracy: 0.9815
Epoch 85/200
11/11 [==============================] - 0s 1ms/step - loss: 0.0898 - accuracy: 1.0000
Epoch 86/200
11/11 [==============================] - 0s 1ms/step - loss: 0.1378 - accuracy: 1.0000
Epoch 87/200
11/11 [==============================] - 0s 2ms/step - loss: 0.1457 - accuracy: 0.9444
Epoch 88/200
11/11 [==============================] - 0s 1ms/step - loss: 0.1178 - accuracy: 0.9630
Epoch 89/200
11/11 [==============================] - 0s 1ms/step - loss: 0.0790 - accuracy: 0.9815
Epoch 90/200
11/11 [==============================] - 0s 2ms/step - loss: 0.1135 - accuracy: 0.9815
Epoch 91/200
11/11 [==============================] - 0s 1ms/step - loss: 0.0873 - accuracy: 1.0000
Epoch 92/200
11/11 [==============================] - 0s 2ms/step - loss: 0.0616 - accuracy: 1.0000
Epoch 93/200


11/11 [==============================] - 0s 2ms/step - loss: 0.0277 - accuracy: 1.0000
Epoch 165/200
11/11 [==============================] - 0s 1ms/step - loss: 0.0522 - accuracy: 0.9815
Epoch 166/200
11/11 [==============================] - 0s 1ms/step - loss: 0.0312 - accuracy: 1.0000
Epoch 167/200
11/11 [==============================] - 0s 2ms/step - loss: 0.0245 - accuracy: 0.9815
Epoch 168/200
11/11 [==============================] - 0s 1ms/step - loss: 0.0183 - accuracy: 1.0000
Epoch 169/200
11/11 [==============================] - 0s 1ms/step - loss: 0.0972 - accuracy: 0.9630
Epoch 170/200
11/11 [==============================] - 0s 2ms/step - loss: 0.0254 - accuracy: 1.0000
Epoch 171/200
11/11 [==============================] - 0s 1ms/step - loss: 0.0321 - accuracy: 1.0000
Epoch 172/200
11/11 [==============================] - 0s 2ms/step - loss: 0.0267 - accuracy: 1.0000
Epoch 173/200
11/11 [==============================] - 0s 1ms/step - loss: 0.0055 - accuracy: 1.0000
Epoc

In [18]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np
from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [19]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    #return res,results,return_list
    return return_list

In [20]:
sentence="hello"
predict_class(sentence, model)

[{'intent': 'greeting', 'probability': '0.9996785'}]

In [21]:
classes

['CareerOpportunities',
 'applying',
 'bootname',
 'classsize',
 'coordinator',
 'goodbye',
 'greeting',
 'keycourses',
 'options',
 'subjects',
 'thanks']

In [22]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result
def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

In [ ]:
#Creating Simple GUI with tkinter
import tkinter
from tkinter import *
def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "👤: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Helvetica", 14 ))
        res = chatbot_response(msg)
        ChatLog.insert(END, "🤖: " + res + '\n\n')
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
base = Tk()
base.title("MBD Mater Program Assistant 🤖")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)
#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="190", font="Helvetica",)
ChatLog.config(state=DISABLED)
#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview,)
ChatLog['yscrollcommand'] = scrollbar.set
#Create Button to send message
SendButton = Button(base, font=("Helvetica",14,'bold'), text="Send ➢", width="12", height=5,
                    bd=5, bg="green", activebackground="green",fg='green',
                    command= send )
#Create the box to enter message
EntryBox = Text(base, bd=5, bg="white",width="29", height="5", font="Helvetica")
#EntryBox.bind("<Return>", send)
#Place all components on the screen
scrollbar.place(x=376,y=6, height=386) #386
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)
base.mainloop()

In [43]:
from tkinter import Tk, font
root = Tk()
font.families()

('Academy Engraved LET',
 'Al Bayan',
 'Al Nile',
 'Al Tarikh',
 'American Typewriter',
 'Andale Mono',
 'Apple Braille',
 'Apple Chancery',
 'Apple Color Emoji',
 'Apple SD Gothic Neo',
 'Apple Symbols',
 'AppleGothic',
 'AppleMyungjo',
 'Arial',
 'Arial Black',
 'Arial Hebrew',
 'Arial Hebrew Scholar',
 'Arial Narrow',
 'Arial Rounded MT Bold',
 'Arial Unicode MS',
 'Avenir',
 'Avenir Next',
 'Avenir Next Condensed',
 'Ayuthaya',
 'Baghdad',
 'Bangla MN',
 'Bangla Sangam MN',
 'Baskerville',
 'Beirut',
 'Big Caslon',
 'Bodoni 72',
 'Bodoni 72 Oldstyle',
 'Bodoni 72 Smallcaps',
 'Bodoni Ornaments',
 'Bradley Hand',
 'Brush Script MT',
 'Chalkboard',
 'Chalkboard SE',
 'Chalkduster',
 'Charter',
 'Cochin',
 'Comic Sans MS',
 'Copperplate',
 'Corsiva Hebrew',
 'Courier',
 'Courier New',
 'Damascus',
 'DecoType Naskh',
 'Devanagari MT',
 'Devanagari Sangam MN',
 'Didot',
 'DIN Alternate',
 'DIN Condensed',
 'Diwan Kufi',
 'Diwan Thuluth',
 'Euphemia UCAS',
 'Farah',
 'Farisi',
 'Futura',